In [2]:
import requests

client_id = '77e238f16d4c4ad696c0c13ebf18d6ba'
client_secret = '2192639a6a8f477aa1fe31e915922386'

auth_url = 'https://accounts.spotify.com/api/token'
data = {
    'grant_type': 'client_credentials',
    'client_id': client_id,
    'client_secret': client_secret,
}

auth_response = requests.post(auth_url, data=data)
access_token = auth_response.json().get('access_token')
access_token

'BQA5tXRf637q0D2EwL5Lnoyp7ZOZ5LGuTzpv6QwoF_h9q3Oj6xWiOyWXA4lSxOwdfadSBC7yBkCCbQyGlBpD1TFBNnyCrRCGUm8gxFgYFHYV6Vv9L04r'

In [5]:
import csv
import requests

# Set up your access token
access_token = access_token

# Set up the API endpoint
endpoint = "https://api.spotify.com/v1/search"

# Set up the query parameters
query_params = {
    "q": "genre:'Romantic'",
    "type": "track",
    "limit": 50  # Adjust the limit as per your needs
}

# Set up the request headers with the access token
headers = {
    "Authorization": f"Bearer {access_token}"
}

# Send the initial GET request to the Spotify API
response = requests.get(endpoint, params=query_params, headers=headers)

# Parse the response JSON
response_json = response.json()
print(response_json)
# Extract the relevant data from the response
tracks = response_json["tracks"]["items"]

# Open the CSV file in write mode
with open("songs.csv", mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)

    # Write the header row
    writer.writerow(["Song Name", "Artist(s)", "Album"])

    # Write the song details to the CSV file
    for track in tracks:
        song_name = track["name"]
        artists = ", ".join([artist["name"] for artist in track["artists"]])
        album_name = track["album"]["name"]
        writer.writerow([song_name, artists, album_name])

# Check if there are more pages of results
while response_json["tracks"]["next"]:
    next_url = response_json["tracks"]["next"]

    # Send the next GET request to the Spotify API for the next page
    response = requests.get(next_url, headers=headers)

    # Parse the response JSON
    response_json = response.json()

    # Extract the relevant data from the response
    tracks = response_json["tracks"]["items"]

    # Append the song details to the CSV file
    with open("songs.csv", mode="a", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        for track in tracks:
            song_name = track["name"]
            artists = ", ".join([artist["name"] for artist in track["artists"]])
            album_name = track["album"]["name"]
            writer.writerow([song_name, artists, album_name])


{'tracks': {'href': 'https://api.spotify.com/v1/search?query=genre%3A%27Romantic%27&type=track&offset=0&limit=50', 'items': [{'album': {'album_type': 'album', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/4bthk9UfsYUYdcFyqxmSUU'}, 'href': 'https://api.spotify.com/v1/artists/4bthk9UfsYUYdcFyqxmSUU', 'id': '4bthk9UfsYUYdcFyqxmSUU', 'name': 'Tears For Fears', 'type': 'artist', 'uri': 'spotify:artist:4bthk9UfsYUYdcFyqxmSUU'}], 'available_markets': ['AD', 'AE', 'AG', 'AL', 'AM', 'AO', 'AR', 'AT', 'AU', 'AZ', 'BA', 'BB', 'BD', 'BE', 'BF', 'BG', 'BH', 'BI', 'BJ', 'BN', 'BO', 'BR', 'BS', 'BT', 'BW', 'BY', 'BZ', 'CA', 'CD', 'CG', 'CH', 'CI', 'CL', 'CM', 'CO', 'CR', 'CV', 'CW', 'CY', 'CZ', 'DE', 'DJ', 'DK', 'DM', 'DO', 'DZ', 'EC', 'EE', 'EG', 'ES', 'ET', 'FI', 'FJ', 'FM', 'FR', 'GA', 'GB', 'GD', 'GE', 'GH', 'GM', 'GN', 'GQ', 'GR', 'GT', 'GW', 'GY', 'HK', 'HN', 'HR', 'HT', 'HU', 'ID', 'IE', 'IL', 'IN', 'IQ', 'IS', 'IT', 'JM', 'JO', 'JP', 'KE', 'KG', 'KH', 'KI', 'KM', 

In [6]:
import pandas as pd
df = pd.read_csv('songs.csv', encoding= 'unicode_escape')

df.head()

,Song Name,Artist,Album
0,Everybody Wants To Rule The World,Tears For Fears,Songs From The Big Chair (Super Deluxe Edition)
1,Take on Me,a-ha,Hunting High and Low
2,Don't You (Forget About Me),Simple Minds,Once Upon A Time (Super Deluxe)
3,Sweet Dreams (Are Made of This) - Remastered,"Eurythmics, Annie Lennox, Dave Stewart",Sweet Dreams (Are Made Of This)
4,Wake Me Up Before You Go-Go,Wham!,Make It Big


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Song Name  1000 non-null   object
 1   Artist     1000 non-null   object
 2   Album      1000 non-null   object
dtypes: object(3)
memory usage: 23.6+ KB


In [9]:
df = df.drop_duplicates(subset=['Song Name', 'Artist'], keep='first')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 934 entries, 0 to 999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Song Name  934 non-null    object
 1   Artist     934 non-null    object
 2   Album      934 non-null    object
dtypes: object(3)
memory usage: 29.2+ KB


In [10]:
df.to_csv('love_songs.csv', index=False)

In [11]:
def get_song_details(access_token, song_name, artist, album):
    search_url = 'https://api.spotify.com/v1/search'
    headers = {
        'Authorization': 'Bearer ' + access_token
    }
    params = {
        'q': f'track:"{song_name}" artist:"{artist}" album:"{album}"',
        'type': 'track',
        'limit': 1
    }
    response = requests.get(search_url, headers=headers, params=params)
    response_data = response.json()
    print(response_data)
    return response_data['tracks']['items'][0] if response_data['tracks']['items'] else None


In [12]:
input_file = 'love_songs.csv'
output_file = 'love_songs_with_details.csv'

# Open the input and output files
with open(input_file, 'r') as csv_file, open(output_file, 'w', newline='') as output_csv:
    reader = csv.DictReader(csv_file)
    fieldnames = reader.fieldnames + ['spotify_id', 'duration_ms', 'popularity']
    writer = csv.DictWriter(output_csv, fieldnames=fieldnames)
    writer.writeheader()

    # Get the access token

    # Iterate over each song in the input file
    for row in reader:
        song_name = row['Song Name']
        artist = row['Artist']
        album = row['Album']

        # Get the song details from Spotify API
        song_details = get_song_details(access_token, song_name, artist, album)

        if song_details:
            # Extract the relevant details
            spotify_id = song_details['id']
            duration_ms = song_details['duration_ms']
            popularity = song_details['popularity']

            # Write the updated row to the output file
            row['spotify_id'] = spotify_id
            row['duration_ms'] = duration_ms
            row['popularity'] = popularity
            writer.writerow(row)


{'tracks': {'href': 'https://api.spotify.com/v1/search?query=track%3A%22Everybody+Wants+To+Rule+The+World%22+artist%3A%22Tears+For+Fears%22+album%3A%22Songs+From+The+Big+Chair+%28Super+Deluxe+Edition%29%22&type=track&offset=0&limit=1', 'items': [{'album': {'album_type': 'album', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/4bthk9UfsYUYdcFyqxmSUU'}, 'href': 'https://api.spotify.com/v1/artists/4bthk9UfsYUYdcFyqxmSUU', 'id': '4bthk9UfsYUYdcFyqxmSUU', 'name': 'Tears For Fears', 'type': 'artist', 'uri': 'spotify:artist:4bthk9UfsYUYdcFyqxmSUU'}], 'available_markets': ['AD', 'AE', 'AG', 'AL', 'AM', 'AO', 'AR', 'AT', 'AU', 'AZ', 'BA', 'BB', 'BD', 'BE', 'BF', 'BG', 'BH', 'BI', 'BJ', 'BN', 'BO', 'BR', 'BS', 'BT', 'BW', 'BY', 'BZ', 'CA', 'CD', 'CG', 'CH', 'CI', 'CL', 'CM', 'CO', 'CR', 'CV', 'CW', 'CY', 'CZ', 'DE', 'DJ', 'DK', 'DM', 'DO', 'DZ', 'EC', 'EE', 'EG', 'ES', 'ET', 'FI', 'FJ', 'FM', 'FR', 'GA', 'GB', 'GD', 'GE', 'GH', 'GM', 'GN', 'GQ', 'GR', 'GT', 'GW', 'GY',

KeyError: 'tracks'

In [13]:
def get_audio_features(access_token, track_id):
    audio_features_url = f'https://api.spotify.com/v1/audio-features/{track_id}'
    headers = {
        'Authorization': 'Bearer ' + access_token
    }
    response = requests.get(audio_features_url, headers=headers)
    response_data = response.json()
    return response_data


In [14]:
input_file = 'love_songs_with_details.csv'
output_file = 'love_songs_with_audio_features.csv'

# Open the input and output files
with open(input_file, 'r') as csv_file, open(output_file, 'w', newline='') as output_csv:
    reader = csv.DictReader(csv_file)
    fieldnames = reader.fieldnames + ['acousticness', 'instrumentalness', 'liveness', 'loudness', 'danceability', 'energy', 'tempo', 'valence']
    writer = csv.DictWriter(output_csv, fieldnames=fieldnames)
    writer.writeheader()


    # Iterate over each song in the input file
    for row in reader:
        song_id = row['spotify_id']

        # Get the audio features from Spotify API
        audio_features = get_audio_features(access_token, song_id)

        # Extract the relevant audio features
        acousticness = audio_features['acousticness']
        instrumentalness = audio_features['instrumentalness']
        liveness = audio_features['liveness']
        loudness = audio_features['loudness']
        danceability = audio_features['danceability']
        energy = audio_features['energy']
        tempo = audio_features['tempo']
        valence = audio_features['valence']

        # Update the row with the audio features
        row['acousticness'] = acousticness
        row['instrumentalness'] = instrumentalness
        row['liveness'] = liveness
        row['loudness'] = loudness
        row['danceability'] = danceability
        row['energy'] = energy
        row['tempo'] = tempo
        row['valence'] = valence

        # Write the updated row to the output file
        writer.writerow(row)


In [15]:
import pickle
loaded_model = pickle.load(open('rf_model.sav', 'rb'))

df = pd.read_csv('love_songs_with_audio_features.csv', encoding= 'unicode_escape')
df = df.drop(['spotify_id', 'Artist', 'Album', 'Song Name'], axis=1)
result = loaded_model.predict(df)

#add the result to the dataframe
df['result'] = result

In [16]:
df

,duration_ms,popularity,acousticness,instrumentalness,liveness,loudness,danceability,energy,tempo,valence,result
0,251488,88,0.34700,0.003890,0.1040,-12.095,0.645,0.795,112.067,0.535,1
1,225280,88,0.01800,0.001250,0.0928,-7.638,0.573,0.902,84.412,0.876,-1
2,231333,81,0.27100,0.000000,0.0607,-11.893,0.620,0.573,81.548,0.897,1
3,194120,73,0.16500,0.000000,0.1050,-5.851,0.727,0.834,131.700,0.816,-1
4,287306,77,0.66400,0.000003,0.2590,-7.049,0.450,0.656,106.610,0.782,1
...,...,...,...,...,...,...,...,...,...,...,...
158,253253,58,0.00332,0.000014,0.0986,-8.010,0.660,0.806,122.041,0.944,-1
159,286106,58,0.06060,0.098800,0.0646,-11.238,0.718,0.543,115.522,0.370,-1
160,268200,56,0.04390,0.002320,0.0613,-20.208,0.579,0.223,159.696,0.532,1
161,268546,59,0.08800,0.156000,0.0755,-11.687,0.635,0.684,120.269,0.892,-1


In [ ]:
df.to_csv("big_train.csv", index=False)